# 1.Import Libarries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2. Import Data

In [ ]:
df = pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv')
df.head()

In [ ]:
df.label.value_counts(ascending = False)

In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Data Cleaning and preprocessing

In [ ]:
#Data have some inpredictiable and useless character and also there are puncutations are present so will remove using Natural language preprocessing library and also we will remove stopwords.
lem=WordNetLemmatizer()

corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['tweet'][i]) #Instead of Punctuations it just replacing with " ".
    review = review.lower() #Lowering each sentence
    review = review.split() #After lowering and replacing puntucation will split the words
    review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')] # Here we will drop words which are present in stopwords like ['can't,they,we,here,there].
    review = ' '.join(review)
    corpus.append(review) #All sentences are collect in corpus.

In [ ]:
corpus[7]

In [ ]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer #Using TFIDF vectorizer we will convert document in vector form.
cv = TfidfVectorizer(max_features = 7777,ngram_range = (1,3))
x= cv.fit_transform(corpus).toarray()
x

In [ ]:
x.shape

In [ ]:
y = df.loc[:,['label']]
y

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)
print('Training data:{}'.format(X_train.shape))
print('Test data:{}'.format(X_test.shape))

In [ ]:
cv.get_feature_names()[:20]

In [ ]:
cv.get_params()

In [ ]:
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

count_df

# 4.Built model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(X_train,y_train)
y_predicted = clf.predict(X_test)
score = clf.score(X_test,y_test)
print(score)

In [ ]:
from sklearn.metrics import confusion_matrix

#Confusion Matrix
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_predicted)
np.set_printoptions(precision=2)
cnf_matrix

In [ ]:
test = pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv')
test.head()

In [ ]:
text_corpus = []
for i in range(0, len(test)):
    review = re.sub('[^a-zA-Z]', ' ', test['tweet'][i])
    review = review.lower()
    review = review.split()
    review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    text_corpus.append(review)

In [ ]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
test_cv = TfidfVectorizer(max_features = 7777,ngram_range = (1,3))
x= test_cv.fit_transform(text_corpus).toarray()
x

In [ ]:
test_df = pd.DataFrame(x, columns=test_cv.get_feature_names())

test_df

In [ ]:
test_y = clf.predict(test_df)
test_y